Proyecto jiji

In [ ]:
!pip install kagglehub

In [ ]:
import nltk
import numpy as np
from nltk.stem.snowball import SnowballStemmer
nltk.download('punkt')
import regex as re

[nltk_data] Downloading package punkt to C:\Users\Ce
[nltk_data]     mar\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
stemmer = SnowballStemmer('english')
import os
import pandas as pd

In [ ]:
import kagglehub
path = kagglehub.dataset_download("imuhammad/audio-features-and-lyrics-of-spotify-songs")
print("Path to dataset files:", path)



Path to dataset files: C:\Users\Ce mar\.cache\kagglehub\datasets\imuhammad\audio-features-and-lyrics-of-spotify-songs\versions\1


In [ ]:
with open("stopwords-en.txt", encoding="latin1") as file:
   stoplist = [line.rstrip().lower() for line in file]
stoplist += ['?', '-', '.', ':', ',', '!', ';']

def preprocesamiento(texto, stemming=True):
  words = []
  if not isinstance(texto, str):
    return words
  texto = texto.lower()
  texto = re.sub(r'[^a-zA-Z0-9_À-ÿ]', ' ', texto)
  # tokenizar
  words = nltk.word_tokenize(texto, language='spanish')
  # filtrar stopwords
  words = [word for word in words if word not in stoplist]
  # reducir palabras (stemming)
  if stemming:
      words = [stemmer.stem(word) for word in words]
  return words


In [ ]:
lista_ = os.listdir(path)
songs = os.path.join(path, lista_[0])
print("Path de songs: ",songs)
dataset = pd.read_csv(songs)

Path de songs:  C:\Users\Ce mar\.cache\kagglehub\datasets\imuhammad\audio-features-and-lyrics-of-spotify-songs\versions\1\spotify_songs.csv


In [ ]:
fila_5 = dataset.iloc[4, 3]
print(fila_5)



Get up out of my business You don't keep me from turning up 모두 다 여긴 Witness 넌 바른 척하는 Criminal 찔리는 걸 You feel it 너 네가 뭔데 왜 솔직하지 못해 넌 다를 건 대체 뭔데 aye Bad boy 그래 난 Bad boy 생각해 네가 하고픈 대로 That boy 그래 난 That boy 모르면 믿고 싶은 대로 Nothing ‘bout me nada you don't know Nothing ‘bout me not a thing 욕을 하는 이윤 다 있는 거야 나 아닌 너 안에 Cause we Party yeah we party we don't worry ‘bout the drama no Body Everybody 우린 아침까지 Wilding up 신경 쓸건 따로 아 몰라 오늘 다 집어치워 눈치 보지 말고 Volume up up up We go dumb dumb dumb dumb 누가 뭐래도 Litty dumb Litty dumb litty Bang the drum 갈 때까지 Baby stupid Stupid dumb dumb dumb dumb 누가 뭐래도 Litty dumb Litty dumb litty Bang the drum 갈 때까지 Baby we go 몰입하니 눈 몰리는 이치 시시콜콜 무료함을 완충해 내키면 곧장 Dumb litty 이 노래가 흐름 Come get it Drippin' 여기 주도권을 Grip than 거리낌 허물어 Let's go party 돌변함은 때 때에 따라 아무렴 어때 까무러치게 더불어 즐겨 사는 거지 My way follow nobody Party yeah we party we don't worry ‘bout the drama no Body Everybody 우린 아침까지 Wilding up 신경 쓸건 따로 아 몰라 오늘 다 집어치워 터지게 더 올려 Volume up up up We go dumb dumb dumb dumb 누가 뭐래도 Litty dumb

In [ ]:
# dataset = dataset.head(l)
dataset = dataset.head(20)
#Procesar los documentos en pares
position_text=3
pairs = []
for i, row in dataset.iterrows():
    words = preprocesamiento(row.iloc[position_text])
    for text in words:
        pairs.append((i, text))

pairs_df = pd.DataFrame(pairs, columns=['Index', 'ProcessedText'])
pairs_df.head(len(pairs_df))


print(pairs_df[pairs_df["Index"] == 0])

     Index ProcessedText
0        0        minsan
1        0          nang
2        0           ako
3        0    napalingon
4        0         hindi
..     ...           ...
106      0         ngiti
107      0          akin
108      0            ay
109      0  nakapagbigay
110      0        pansin

[111 rows x 2 columns]


In [ ]:
import pickle
def Lista_To_Diccionario(dir_bloque):
    
    bloque_dict = {}
    try:
        with open(dir_bloque, 'rb') as f:
            bloque = pickle.load(f)
        if len(bloque) != 0:
            print("bloque es: ",bloque)
            for term, posting in bloque:
                term_dict = {}
                for doc, tf in posting:
                    term_dict[doc] = tf
                bloque_dict[term] = term_dict
    except EOFError:
        print(f"El archivo {dir_bloque} está vacío o incompleto.")
    return bloque_dict

# print(Lista_To_Diccionario("index_blocks/block_1.pkl"))

In [ ]:
import os
import pickle
import sys
import math
import heapq

class SPIMI:
    def __init__(self, index_dir="index_blocks"):
        self.index_dir = index_dir  
        self.block_counter = 0      
        self.doc_count = 0  
        self.doc_ids = set()         
        self.idf = {}
        self.length = {}
        self.disk_limit = 4000  
        self.totalLevels = 0 
        self.currentL = 0
        self.cuntermerged = 0 

        if not os.path.exists(self.index_dir):
            os.makedirs(self.index_dir)

    def spimi_invert(self, token_stream):
        dictionary = {}
        for _, row in token_stream.iterrows():
            doc_id = row['Index']
            token = row['ProcessedText']
            self.doc_ids.add(doc_id)
            if token not in dictionary:
                dictionary[token] = {}  
            
            if doc_id not in dictionary[token]:
                dictionary[token][doc_id] = 1  
            else:
                dictionary[token][doc_id] += 1  
            dictionary_size = sys.getsizeof(dictionary)
            if dictionary_size >= self.disk_limit:
                self.write_block_to_disk(dictionary, level=0)
                dictionary.clear()
                
        if dictionary:
            self.write_block_to_disk(dictionary, level=0)

        self.totalLevels = math.ceil(math.log2(self.block_counter))
        print("La cantidad de niveles serán", self.totalLevels)
        print("Cantidad de bloques generados", self.block_counter)

        ##
        self.load_index()
        self.calculate_idf()
        
    def write_block_to_disk(self, dictionary, level):
        sorted_terms = dict(sorted(dictionary.items())) 
        # block_{it}
        file_path = os.path.join(self.index_dir, f"block_{self.block_counter}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(sorted_terms, f)
        
        self.block_counter += 1

    def load_block(self, filepath):
        with open(filepath, "rb") as f:
            return pickle.load(f)

    def save_merged_block(self, merged_data, level):
        file_path = os.path.join(self.index_dir, f"block_{self.block_counter}.pkl")
        with open(file_path, "wb") as f:
            pickle.dump(merged_data, f)
        self.cuntermerged += 1

    def retrieval(self, query, k):
        self.load_index()  # Cargar el índice
        print("El index es:", self.index)
        N = len(self.doc_ids)  # Número total de documentos
        scores = [0] * N  # Puntuaciones para cada documento
        tf_query = {}  # Almacenaremos el TF de los términos de la consulta
        terms = preprocesamiento(query)  # Función de preprocesamiento para tokenizar

        # Calcular el TF-IDF del query
        for term in terms:
            if term in tf_query:
                tf_query[term] += 1
            else:
                tf_query[term] = 1
        print("tf_query es: ", tf_query)

        tfidf_query = {}
        for term, tf in tf_query.items():
            if term in self.idf:
                tfidf_query[term] = math.log10(1 + tf) * self.idf[term]
        print("tfidf_query es: ", tfidf_query)
        norm_query = math.sqrt(sum(w_tq**2 for w_tq in tfidf_query.values()))  # Normalización del query

        # Aplicar similitud de coseno: Calculamos el puntaje para cada documento
        for term, w_tq in tfidf_query.items():
            if term in self.index:
                for doc, tf_td in self.index[term]:
                    w_td =tf_td* self.idf[term]
                    # print("self.index", term, "es", self.index[term])

                    scores[doc] += w_td * w_tq #Producto punto

        # Normalizar las puntuaciones de los documentos
        for d in range(N):
            if self.length.get(d, 0) != 0:
                scores[d] /= (self.length.get(d, 1) * norm_query)  # Normalización documento y consulta

        # Ordenar las puntuaciones en orden descendente
        result = sorted(enumerate(scores), key=lambda x: x[1], reverse=True)
        print("result es", result)

        # Devolver los k documentos más relevantes (top-k)
        return result[:k]
    
    def calculate_idf(self):
        for term, postings in self.index.items():
            # Número de documentos que contienen el término
            doc_freq = len(postings)
            # Calcular IDF y almacenar
            self.idf[term] = math.log10(len(self.doc_ids) / (1 + doc_freq))
        # Calcular la longitud de cada documento
        for term, postings in self.index.items():
            for doc_id, tf in postings:
                if doc_id not in self.length:
                    self.length[doc_id] = 0
                # Sumar los TF-IDF al cuadrado para la longitud del documento
                self.length[doc_id] += (tf * self.idf[term]) ** 2
    
        # Tomar la raíz cuadrada para completar la longitud de cada documento
        for doc_id in self.length:
            self.length[doc_id] = math.sqrt(self.length[doc_id])

    def load_index(self):
        self.index = {}
        files = sorted(os.listdir(self.index_dir))
        for file in files:
            if file.endswith(".pkl"):
                block = self.load_block(os.path.join(self.index_dir, file))
                for term, postings in block.items():
                    if term not in self.index:
                        self.index[term] = []
                    for doc_id, tf in postings.items():
                        
                        self.index[term].append((doc_id, math.log10(1+ tf)))
 
    def load_block(self, filepath):
        """ Cargar un bloque del índice desde un archivo. """
        with open(filepath, "rb") as f:
            return pickle.load(f)




s = SPIMI()  
s.spimi_invert(pairs_df)


NameError: name 'pairs_df' is not defined

In [ ]:
def OrdenarPorBloques(dir_blocks, n_blocks):
    for i in range(n_blocks):
        file_path = os.path.join(dir_blocks, 'block_{}.pkl'.format(i))
        with open(file_path, 'rb') as f:
            tuplas_ordenadas = pickle.load(f)
        tuplas_ordenadas = sorted(list(tuplas_ordenadas.items()), key=lambda x: x[0])
        tuplas_ordenadas = [par for par in tuplas_ordenadas if not par[0].isdigit()]
        tuplas_ordenadas = [(term[0], list(term[1].items())) for term in tuplas_ordenadas]
        with open(file_path, 'wb') as f:
            pickle.dump(tuplas_ordenadas, f)

def mergeSortAux(dir_bloques, l, r):
    if l == r:
        bloque = leer_bloque(dir_bloques, l)
        unicos = set()    
        for par in bloque:
            unicos.add(par[0])
        return list(unicos)
    
    if (l < r):
        mid = int(math.ceil((r + l)/2.0))
        unique_l = mergeSortAux(dir_bloques, l, mid - 1)
        unique_r = mergeSortAux(dir_bloques, mid, r)
        unicos = set()    
        for term in unique_l:
            unicos.add(term)
        for term in unique_r:
            unicos.add(term)
        unicos = list(unicos)
        merge_v2(dir_bloques, l, r, mid, len(unicos))
        return list(unicos)
        
    return []

def escribir_bloque(dir_bloques, block, idx_insert_block, buffer_limit = 2000):
    with open(os.path.join(dir_bloques, "block_{}_v2.pkl".format(idx_insert_block)), 'wb') as f:
        pickle.dump(block, f)    
def leer_bloque(dir_bloques, it):
    file_path = os.path.join(dir_bloques, f"block_{it}.pkl")
    with open(file_path, "rb") as f:
        buffer = pickle.load(f)
    return buffer
def merge_v2(dir_bloques, l, r, mid, num_terms):
    idx_insert_block = l
    new_block = []
    mezclar_n_bloques = r - l + 1
    unique_terms_per_block = int(math.ceil(num_terms/mezclar_n_bloques))
    unique_terms_current_block = 0

    it_l = l
    it_r = mid
    term_dic_l = leer_bloque(dir_bloques, it_l)
    term_dic_r = leer_bloque(dir_bloques, it_r)
    
    idx_term_l = 0
    idx_term_r = 0

    idx_doc_l = 0
    idx_doc_r = 0
    new_block = []
    while(it_l < mid and it_r < r + 1):
        print(f"Toma 2 bloques {it_l} y {it_r} | idx_term_l: ", idx_term_l, "| len(term_dic_l)", len(term_dic_l), "| idx_term_r: ", idx_term_r, "| len(term_dic_r)", len(term_dic_r))
        while(idx_term_l < len(term_dic_l) and idx_term_r < len(term_dic_r)): # moverme entre palabras de dos bloques
            # print("Current term_dic_l", term_dic_l)
            # print("Current term_dic_r", term_dic_r)
            # print(f"Toma 2 terminos {term_dic_l[idx_term_l][0]} y {term_dic_r[idx_term_r][0]}")
            new_term = []
            if(term_dic_l[idx_term_l][0] < term_dic_r[idx_term_r][0]):
                new_term = term_dic_l[idx_term_l]
                idx_term_l += 1
            elif(term_dic_l[idx_term_l][0] > term_dic_r[idx_term_r][0]):
                new_term = term_dic_r[idx_term_r]
                idx_term_r += 1
            else:
                idx_doc_l = 0
                idx_doc_r = 0
                while(idx_doc_l < len(term_dic_l[idx_term_l][1]) and idx_doc_r < len(term_dic_r[idx_term_r][1])):
                    print(f"Toma 2 terminos iguales con tf = {term_dic_l[idx_term_l][1]} y {term_dic_r[idx_term_r][1]}")
                    if term_dic_l[idx_term_l][1][idx_doc_l][0] > term_dic_r[idx_term_r][1][idx_doc_r][0]:
                        pushear_doc = term_dic_r[idx_term_r][1][idx_doc_r]
                        idx_doc_r += 1
                    elif term_dic_l[idx_term_l][1][idx_doc_l][0] < term_dic_r[idx_term_r][1][idx_doc_r][0]:
                        pushear_doc = term_dic_l[idx_term_l][1][idx_doc_l]
                        idx_doc_l += 1
                    else:
                        pushear_doc = (term_dic_l[idx_term_l][1][idx_doc_l][0], term_dic_l[idx_term_l][1][idx_doc_l][1] + term_dic_r[idx_term_r][1][idx_doc_r][1])
                        idx_doc_l += 1
                        idx_doc_r += 1
                    print("pushear_doc: ", pushear_doc)
                    new_term.append(pushear_doc)
                while(idx_doc_l < len(term_dic_l[idx_term_l][1])):
                    print(f"ya no hay documentos de derecha, rellena con izquierda {idx_doc_l}")
                    pushear_doc = term_dic_l[idx_term_l][1][idx_doc_l]
                    idx_doc_l += 1
                    new_term.append(pushear_doc)
                while(idx_doc_r < len(term_dic_r[idx_term_r][1])):
                    print(f"ya no hay documentos de izquierda, rellena con derecha {idx_doc_l}")
                    pushear_doc = term_dic_r[idx_term_r][1][idx_doc_r]
                    idx_doc_r += 1
                    new_term.append(pushear_doc)
                new_term = (term_dic_l[idx_term_l][0], new_term)
                idx_term_r += 1
                idx_term_l += 1
            new_block.append(new_term)
            
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
        if(len(term_dic_l) == idx_term_l):
            if (it_l < mid - 1):
                it_l += 1
                term_dic_l = leer_bloque(dir_bloques, it_l)
                idx_term_l = 0
                idx_doc_l = 0
                continue
            else:
                break
        if(len(term_dic_r) == idx_term_r):
            if (it_r < r):
                it_r += 1
                term_dic_r = leer_bloque(dir_bloques, it_r)
                idx_term_r = 0
                idx_doc_r = 0
                continue
            else:
                break
        if(it_l == mid | it_r == r + 1):
            break
    while(it_l < mid):
        print(f"Se acabaron los bloques de derecha, llena solo izquierda {it_l}")
        term_dic_l = leer_bloque(dir_bloques, it_l)
        while(idx_term_l < len(term_dic_l)):
            new_block.append(term_dic_l[idx_term_l])
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
            idx_term_l += 1
        idx_term_l = 0
        it_l += 1
    while(it_r < r + 1):
        print(f"Se acabaron los bloques de izquierda, llena solo derecha {it_r}")
        term_dic_r = leer_bloque(dir_bloques, it_r)
        while(idx_term_r < len(term_dic_r)):
            new_block.append(term_dic_r[idx_term_r])
            unique_terms_current_block += 1
            if (unique_terms_current_block == unique_terms_per_block):
                escribir_bloque(dir_bloques, new_block, idx_insert_block)
                unique_terms_current_block = 0
                idx_insert_block += 1
                new_block = []
            idx_term_r += 1
        idx_term_r = 0
        it_r += 1

    while(idx_insert_block < r + 1):
        if len(new_block) > 0:
            escribir_bloque(dir_bloques, new_block, idx_insert_block)
        else:
            escribir_bloque(dir_bloques, [], idx_insert_block)
        new_block = []
        idx_insert_block += 1
    idx_insert_block = l
    for idx_archivo in range(l, r + 1):
        nuevo_nombre = os.path.join(dir_bloques, "block_{}.pkl".format(idx_archivo))
        if os.path.exists(nuevo_nombre):
            os.remove(nuevo_nombre)
        os.rename(os.path.join(dir_bloques, "block_{}_v2.pkl".format(idx_archivo)), nuevo_nombre)
def mergeSort(dir_bloques):
    bloques_files_dir = os.listdir(os.path.join('./',dir_bloques))
    # print(bloques_files_dir)
    n = len(bloques_files_dir)
    # n = int(math.exp2(math.floor(math.log2(n)) + 1))
    mergeSortAux(dir_bloques, 0, n - 1)
'''
Recibe la direccion del folder con los diccionarios del spimi,
modifica los archivos y los sobreescribe para crear el índice
invertido con un índice global
'''


'\nRecibe la direccion del folder con los diccionarios del spimi,\nmodifica los archivos y los sobreescribe para crear el índice\ninvertido con un índice global\n'

In [ ]:
n_blocks = s.block_counter
dir_blocks = 'index_blocks'
OrdenarPorBloques(dir_blocks, n_blocks )
mergeSort(dir_blocks)

Toma 2 bloques 1 y 2 | idx_term_l:  0 | len(term_dic_l) 86 | idx_term_r:  0 | len(term_dic_r) 86
Toma 2 terminos iguales con tf = [(2, 1)] y [(3, 2)]
pushear_doc:  (2, 1)
ya no hay documentos de izquierda, rellena con derecha 1
Toma 2 terminos iguales con tf = [(2, 1)] y [(2, 2)]
pushear_doc:  (2, 3)
Toma 2 terminos iguales con tf = [(2, 1)] y [(3, 1)]
pushear_doc:  (2, 1)
ya no hay documentos de izquierda, rellena con derecha 1
Toma 2 terminos iguales con tf = [(2, 1)] y [(2, 1)]
pushear_doc:  (2, 2)
Toma 2 terminos iguales con tf = [(2, 1)] y [(2, 1)]
pushear_doc:  (2, 2)
Toma 2 terminos iguales con tf = [(2, 3)] y [(2, 1)]
pushear_doc:  (2, 4)
Toma 2 terminos iguales con tf = [(2, 1)] y [(2, 1)]
pushear_doc:  (2, 2)
Toma 2 terminos iguales con tf = [(2, 1)] y [(2, 1)]
pushear_doc:  (2, 2)
Toma 2 terminos iguales con tf = [(2, 1)] y [(3, 1)]
pushear_doc:  (2, 1)
ya no hay documentos de izquierda, rellena con derecha 1
Toma 2 terminos iguales con tf = [(2, 1)] y [(2, 1)]
pushear_doc: 

In [ ]:
def InvertirListasDiccionarios(dir_bloques):
    n_bloques = len(os.listdir(os.path.join('./',dir_bloques)))
    for i in range(n_bloques):
        bloque_path = os.path.join(dir_bloques,"block_{}.pkl".format(i))
        with open(bloque_path, 'rb') as f:
            bloque = pickle.load(f)
        bloque_dict = {}

        if len(bloque) != 0:
            for term, poosting_list in bloque:
                poosting_dict = {}
                for doc, tf in poosting_list:
                    poosting_dict[doc] = tf
                bloque_dict[term] = poosting_dict
        with open(bloque_path, 'wb') as f:
            pickle.dump(bloque_dict, f)
InvertirListasDiccionarios("index_blocks")

In [ ]:
def mostrarDocumentos(result):
    print(result)
    for doc, score in result:
        print("Documento", doc, " con similitud: ", score)
    print("_____")   
    


query = dataset.iloc[4, 3]
print("query es", query)
top_k = 5
result = s.retrieval(query, top_k)
mostrarDocumentos(result)

query es Get up out of my business You don't keep me from turning up 모두 다 여긴 Witness 넌 바른 척하는 Criminal 찔리는 걸 You feel it 너 네가 뭔데 왜 솔직하지 못해 넌 다를 건 대체 뭔데 aye Bad boy 그래 난 Bad boy 생각해 네가 하고픈 대로 That boy 그래 난 That boy 모르면 믿고 싶은 대로 Nothing ‘bout me nada you don't know Nothing ‘bout me not a thing 욕을 하는 이윤 다 있는 거야 나 아닌 너 안에 Cause we Party yeah we party we don't worry ‘bout the drama no Body Everybody 우린 아침까지 Wilding up 신경 쓸건 따로 아 몰라 오늘 다 집어치워 눈치 보지 말고 Volume up up up We go dumb dumb dumb dumb 누가 뭐래도 Litty dumb Litty dumb litty Bang the drum 갈 때까지 Baby stupid Stupid dumb dumb dumb dumb 누가 뭐래도 Litty dumb Litty dumb litty Bang the drum 갈 때까지 Baby we go 몰입하니 눈 몰리는 이치 시시콜콜 무료함을 완충해 내키면 곧장 Dumb litty 이 노래가 흐름 Come get it Drippin' 여기 주도권을 Grip than 거리낌 허물어 Let's go party 돌변함은 때 때에 따라 아무렴 어때 까무러치게 더불어 즐겨 사는 거지 My way follow nobody Party yeah we party we don't worry ‘bout the drama no Body Everybody 우린 아침까지 Wilding up 신경 쓸건 따로 아 몰라 오늘 다 집어치워 터지게 더 올려 Volume up up up We go dumb dumb dumb dumb 누가 뭐래도 L